In [1]:
%%HTML
<link rel="stylesheet" href="https://doc.splicemachine.com/jupyter/css/custom.css">

# Introduction

This notebook shows you how to create create custom stored functions and procedures that you can deploy to your Splice Machine database for use from your SQL.

<p class="noteIcon">Currently, stored procedures and functions must be implemented in Java.</p>

## Your Stored Procedure Development Environment

The *environment* in which your stored procedure code is developed may change depending upon your need.  For example, we recommend that you flesh out the actual functionality of your custom code in a local, standalone Splice Machine environment.

You can download our ready-to-use standalone version, or you can build it yourself from our open source code, as described in our [*Getting Started with Splice Machine*](https://www.splicemachine.com/get-started/) web page.

This notebook walks you through the steps to add your own stored procedure or function.

<p class="noteNote">You cannot develop your custom code in a Jupyter notebook; this notebook simply shows you how to so in your own environment.</p>

## Local Development of the Custom Code in an IDE

The steps described here will be for a Mac, but the process is generally the same for Linux. Using an Integrated Development Environment (IDE) is usually the best approach when developing code, since development is straightforward, as is the generation of the desired output; in this case, a JDK 1.8-compatible JAR file. We'll be using IntelliJ IDEA here, though most IDEs should work as well.

Follow these steps to develop your code:

1. Set up Your Project
2. Configure Your Project to Output a JAR File
3. Write Your Code
4. Build Your JAR File


### 1. Set Up Your Project

Follow these steps to set up your project in IntelliJ IDEA:

1. Select *File -> New -> Project...*.
2. Choose *Java with 1.8 as the Project SDK*, then click *Next*.
3. You can optionally decide to create from a template. Click *Next*.
4. Give your project a name and location; e.g. `CompanyACustom1`.
5. If desired, select *File -> New -> Package* to create a new package; e.g. `com.splicemachine.companya`.

### 2. Configure Your Project to Output a JAR file

Follow these steps in IntelliJ IDEA to have your project output a JAR file:

1. Select *File -> Project Structure...*.
2. Select *Artifacts*, then click *+*.
3. Choose *JAR -> From modules with dependencies*.
3. Select *Build on make*, then click *OK*.

### 3. Write Your Code

Now you can write your code:

1. Create a file by right-clicking on your *src* or *package* folder, then select *New -> Java Class*.
2. Add your code. Here's a simple example:

```
package com.splicemachine.companya;

import java.sql.SQLException;
import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.PreparedStatement;


public class Custom1 {


    public static void customIncrement () throws SQLException {
        try (Connection con = DriverManager.getConnection("jdbc:default:connection")) {

            String incrementSQL = "update SPLICE.INCREMENTTEST set i = i + 1";

            try (PreparedStatement ps = con.prepareStatement(incrementSQL)) {
                int resultSet = ps.executeUpdate();
            } catch (SQLException e) {
                con.rollback();
                e.printStackTrace();
                throw e;
            }
        }
    }
}
```

### 4. Build Your JAR File

Finally, build your JAR file and start testing by following these steps:

1. Select *Build -> Artifacts... JAR*.
2. Use your command line interface to navigate to your JAR file; e.g. `out/artifacts/CompanyACustom1_jar/Custom1.jar`.
3. Deploy your JAR file for testing.


## Development Guidelines

Here are some simple best practices to consider when developing and deploying your code:

* It is convenient for code organization to use a package for the code; e.g. `com.splicemachine.yourcompanyname`.
* Notice you don't need a `main` code block in your class. That's because you will be directly calling the class's specific method.
* The example above is a stored procedure, which means that it does not return results. If you're writing a function, it needs to return a result.
* Your procedure or function must throw `SQLException` if it is executing SQL; be sure to use `try`/`catch` blocks similar to the above example to deal with possible run-time exceptions.


## Deploy Your Code in Your Development Environment

To deploy your code into your development environment:

1. Make sure your code is up and running.
2. Carefully edit and execute the commands below to align to your custom procedure and environment:
   
```
CALL SQLJ.INSTALL_JAR('<your-full-path-to>/Custom1.jar', 'SPLICE.Custom1', 0);
CALL SYSCS_UTIL.SYSCS_SET_GLOBAL_DATABASE_PROPERTY('derby.database.classpath', 'SPLICE.custom1');
CREATE PROCEDURE SPLICE.customincrement() PARAMETER STYLE JAVA MODIFIES SQL DATA LANGUAGE JAVA DYNAMIC RESULT SETS 0 EXTERNAL NAME 'com.splicemachine.companya.Custom1.customIncrement';
```

   <p class="noteNote">Perform this operation in your local environment, not in a Jupyter notebook.</p>

## Test Your Code In Your Local Environment

To see your code in action, follow these steps:

1. Set up the table that your code operates on, in your local environment:

```
create table incrementtest (i int);
insert into incrementtest values 1;
```

2. Invoke your code:

```
call customincrement();
select * from incrementtest;
```


## Deploy Your JAR file to Your Cluster
Once you have confirmed that the code works to your satisfaction, you can deploy your JAR file to your Splice Machine cluster on the cloud. Follow the steps in this section:

1. Copy the JAR file to *S3* so that your Service cluster can access it.
2. Deploy the JAR file as you did locally. You can perform this and the next step in a Jupyter notebook, using the `%%SQL` magic.
3. Test and run the custom procedure.

### Copy the JAR file to S3

First make sure the `aws` command tool installed in the local environment where you have your JAR file; if not, [follow the instructions here](http://docs.aws.amazon.com/cli/latest/userguide/installing.html).

Once `aws` is installed, you can run the following commands in your local environment:

```
aws s3 cp <your-file-name> s3://<your-bucket-name>/<your-JAR-file-name>
aws s3api put-object-acl --bucket <your-bucket-name> --grant-read emailAddress="dbaas@splicemachine.com"  --key=<your-JAR-file-name> --region us-east-1
```

### Deploy the JAR File On Your  Cluster

Run the following SQL code to deploy your JAR file on your cluster:

<p class="noteNote">You can run this code in a Jupyter notebook or from the <code>splice></code> command line.

In [9]:
%%sql 
-- installing the jar and defining the procedure
-- edit these steps for your environment
CALL SQLJ.INSTALL_JAR('S3://<your-bucket-name>/<your-jar-file-name>', 'SPLICE.Custom1', 0);
CALL SYSCS_UTIL.SYSCS_SET_GLOBAL_DATABASE_PROPERTY('derby.database.classpath', 'SPLICE.custom1');
CREATE PROCEDURE SPLICE.customincrement() PARAMETER STYLE JAVA MODIFIES SQL DATA LANGUAGE JAVA DYNAMIC RESULT SETS 0 EXTERNAL NAME 'com.splicemachine.companya.Custom1.customIncrement';

### Run Your Code On Your Cluster

You can now run your code using SQL from a Jupyter notebook or from the <code>splice&gt;</code> command line. For example:

In [11]:
%%sql 
create table incrementtest (i int);
insert into incremenenttest values 1;
call customincrement();
select * from incrementtest;